In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('results/lilys_20240711.tsv', sep='\t').set_index('BIOSCAN SUPPLIER SAMPLE NAME')
df.head(1)

,RETENTION INSTRUCTION,COHORT,COUNTRY OF ORIGIN,DATE OF SAMPLE COLLECTION,TAXON ID,COMMON NAME,SAMPLE DESCRIPTION,BIOSCAN CONTROL TYPE
BIOSCAN SUPPLIER SAMPLE NAME,,,,,,,,
CAMP_075_A1,Return to customer after 2 years,CAMP,United Kingdom,2024,32644,unidentified,CAMP_075,NaN


In [3]:
partner_df = pd.read_excel('results/CARM_2407_BIOSCAN_Manifest_V3_0.3.2dev_for_sts.xlsx')
partner_df = partner_df.set_index('SPECIMEN_ID')
partner_df.head(1)

,SERIES,CATCH_LOT,RACK_OR_PLATE_ID,TUBE_OR_WELL_ID,ORGANISM_PART,PRESERVATIVE_SOLUTION,CATCH_SOLUTION,BOTTLE_DIRECTION,DATE_OF_COLLECTION,COUNTRY_OF_COLLECTION,...,ELEVATION,OTHER_INFORMATION,IDENTIFIED_BY,SCIENTIFIC_NAME,TAXON_ID,GAL,SYMBIONT,REGULATORY_COMPLIANCE,HAZARD_GROUP,CONTRIBUTORS
SPECIMEN_ID,,,,,,,,,,,,,,,,,,,,,
CARM_040_A1,1,C043A,CARM_040,A1,WHOLE_ORGANISM,100%_ETHANOL,100%_ETHANOL,N,2023-11-01,UNITED KINGDOM,...,NaN,NaN,NaN,unidentified,32644,SUFFOLK WILDLIFE TRUST (CARLTON MARSHES),TARGET,Y,HG1,Belinda Betts;Suffolk Wildlife Trust;Belinda.B...


In [4]:
patch_samples = df.index.isin(partner_df.index)
patch_samples.sum()

576

In [5]:
df.loc[patch_samples, 'COMMON NAME'].value_counts()

COMMON NAME
unidentified    576
Name: count, dtype: int64

In [6]:
df.loc[patch_samples, 'COMMON NAME'] = partner_df['SCIENTIFIC_NAME']
df.loc[patch_samples, 'COMMON NAME'].value_counts()

COMMON NAME
unidentified    564
blank sample     12
Name: count, dtype: int64

In [7]:
df.loc[patch_samples, 'TAXON ID'].value_counts()

TAXON ID
32644    576
Name: count, dtype: int64

In [8]:
df.loc[patch_samples, 'TAXON ID'] = partner_df['TAXON_ID']
df.loc[patch_samples, 'TAXON ID'].value_counts()

TAXON ID
32644      564
2582415     12
Name: count, dtype: int64

In [12]:
df = df.reset_index()
df.head(1)

,BIOSCAN SUPPLIER SAMPLE NAME,RETENTION INSTRUCTION,COHORT,COUNTRY OF ORIGIN,DATE OF SAMPLE COLLECTION,TAXON ID,COMMON NAME,SAMPLE DESCRIPTION,BIOSCAN CONTROL TYPE
0,CAMP_075_A1,Return to customer after 2 years,CAMP,United Kingdom,2024,32644,unidentified,CAMP_075,NaN


In [14]:
patch_names = patch_samples & \
    (df['COMMON NAME'] == 'blank sample') & \
    ~df['BIOSCAN SUPPLIER SAMPLE NAME'].str.endswith('_G12')
patch_names.sum()

6

In [16]:
df.loc[
    patch_names, 
    'BIOSCAN SUPPLIER SAMPLE NAME'
    ] = 'CONTROL_NEG_LYSATE_' + df['BIOSCAN SUPPLIER SAMPLE NAME']
df.loc[patch_names]

,BIOSCAN SUPPLIER SAMPLE NAME,RETENTION INSTRUCTION,COHORT,COUNTRY OF ORIGIN,DATE OF SAMPLE COLLECTION,TAXON ID,COMMON NAME,SAMPLE DESCRIPTION,BIOSCAN CONTROL TYPE
191,CONTROL_NEG_LYSATE_CARM_032_H12,Return to customer after 2 years,CARM,United Kingdom,2024,2582415,blank sample,CARM_032,NaN
287,CONTROL_NEG_LYSATE_CARM_033_H12,Return to customer after 2 years,CARM,United Kingdom,2024,2582415,blank sample,CARM_033,NaN
383,CONTROL_NEG_LYSATE_CARM_034_H12,Return to customer after 2 years,CARM,United Kingdom,2024,2582415,blank sample,CARM_034,NaN
479,CONTROL_NEG_LYSATE_CARM_035_H12,Return to customer after 2 years,CARM,United Kingdom,2024,2582415,blank sample,CARM_035,NaN
575,CONTROL_NEG_LYSATE_CARM_036_H12,Return to customer after 2 years,CARM,United Kingdom,2024,2582415,blank sample,CARM_036,NaN
671,CONTROL_NEG_LYSATE_CARM_037_H12,Return to customer after 2 years,CARM,United Kingdom,2024,2582415,blank sample,CARM_037,NaN


In [19]:
patch_control = patch_names & df['BIOSCAN SUPPLIER SAMPLE NAME'].str.endswith('_H12')
df.loc[
    patch_names, 
    'BIOSCAN CONTROL TYPE'
    ] = 'lysate negative'
df.loc[patch_samples, 'BIOSCAN CONTROL TYPE'].value_counts(dropna=False)

BIOSCAN CONTROL TYPE
NaN                570
lysate negative      6
Name: count, dtype: int64

In [21]:
df.to_csv('results/lilys_20240711_patched.tsv', sep='\t', index=False)